In [ ]:
!kaggle datasets download -d 'ashikadnan/railway-track-fault-detection-dataset1-rail'

In [ ]:
!unzip railway-track-fault-detection-dataset1-rail.zip

In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report


In [56]:
# Configuration
CONFIG = {
    'IMG_HEIGHT': 299,
    'IMG_WIDTH': 299,
    'BATCH_SIZE': 32,
    'EPOCHS': 50,
    'BASE_DIR': '/kaggle/working/Dataset 1_Rail',
    'TRAIN_DIR': os.path.join('/kaggle/working/Dataset 1_Rail', 'Train'),
    'VALIDATE_DIR': os.path.join('/kaggle/working/Dataset 1_Rail', 'Validiation'), #/kaggle/working/Dataset 1_Rail/Validiation
    'MIN_BATCH_SIZE': 8,  # Minimum batch size to use
    'MAX_BATCH_SIZE': 32  # Maximum batch size to use
}


In [57]:
# Add this after the CONFIG dictionary
import os

# Create directories for outputs
OUTPUT_DIR = 'outputs'
MODEL_DIR = os.path.join(OUTPUT_DIR, 'models')
PLOT_DIR = os.path.join(OUTPUT_DIR, 'plots')

# Create directories if they don't exist
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(PLOT_DIR, exist_ok=True)

In [69]:
def create_data_generators():
    """Create and return data generators for training and validation"""
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    validation_datagen = ImageDataGenerator(rescale=1./255)
    
    # Create generators
    train_generator = train_datagen.flow_from_directory(
        CONFIG['TRAIN_DIR'],
        target_size=(CONFIG['IMG_HEIGHT'], CONFIG['IMG_WIDTH']),
        batch_size=CONFIG['BATCH_SIZE'],
        class_mode='binary',
        shuffle=True,
        seed=42
    )
    
    validation_generator = validation_datagen.flow_from_directory(
        CONFIG['VALIDATE_DIR'],
        target_size=(CONFIG['IMG_HEIGHT'], CONFIG['IMG_WIDTH']),
        batch_size=CONFIG['BATCH_SIZE'],
        class_mode='binary',
        shuffle=True,
        seed=42
    )
    
    return train_generator, validation_generator

In [77]:
def create_model():
    base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(CONFIG['IMG_HEIGHT'], CONFIG['IMG_WIDTH'], 3)
    )
    
    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)  # Reduced from 1024
    x = Dropout(0.7)(x)  # Increased dropout
    predictions = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # Lower learning rate
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model


In [78]:
def train_model():
    try:
        # Create generators with more augmentation
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=40,  # Increased rotation
            width_shift_range=0.3,
            height_shift_range=0.3,
            shear_range=0.3,
            zoom_range=0.3,
            horizontal_flip=True,
            vertical_flip=True,  # Added vertical flip
            fill_mode='nearest'
        )

        validation_datagen = ImageDataGenerator(rescale=1./255)
        
        # Create generators
        train_generator = train_datagen.flow_from_directory(
            CONFIG['TRAIN_DIR'],
            target_size=(CONFIG['IMG_HEIGHT'], CONFIG['IMG_WIDTH']),
            batch_size=16,  # Reduced batch size
            class_mode='binary',
            shuffle=True,
            seed=42
        )
        
        validation_generator = validation_datagen.flow_from_directory(
            CONFIG['VALIDATE_DIR'],
            target_size=(CONFIG['IMG_HEIGHT'], CONFIG['IMG_WIDTH']),
            batch_size=16,  # Reduced batch size
            class_mode='binary',
            shuffle=True,
            seed=42
        )
        
        # Calculate steps
        steps_per_epoch = train_generator.samples // 16
        validation_steps = validation_generator.samples // 16
        
        model = create_model()
        # Define callbacks with adjusted parameters
        callbacks = [
            EarlyStopping(
                monitor='val_loss',
                patience=10,  # Increased patience
                restore_best_weights=True,
                verbose=1
            ),
            ModelCheckpoint(
                'best_model.keras',
                monitor='val_loss',
                save_best_only=True,
                verbose=1
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.2,  # More gradual reduction
                patience=5,
                min_lr=1e-6,
                verbose=1
            )
        ]
        
        # Update CONFIG
        CONFIG.update({
            'BATCH_SIZE': 16,
            'EPOCHS': 100  # Increased epochs
        })
        
        # Train model
        history = model.fit(
            train_generator,
            epochs=CONFIG['EPOCHS'],
            steps_per_epoch=steps_per_epoch,
            validation_data=validation_generator,
            validation_steps=validation_steps,
            callbacks=callbacks,
            verbose=1
        )
        
        return model, history, validation_generator
        
    except Exception as e:
        print(f"Error in train_model: {str(e)}")
        raise

In [79]:
def plot_training_history(history):
    """
    Plot training & validation accuracy and loss values
    
    Args:
        history: Keras history object containing training history
    """
    # Create figure with 2 subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend(loc='lower right')
    ax1.grid(True)
    
    # Plot loss
    ax2.plot(history.history['loss'], label='Training Loss')
    ax2.plot(history.history['val_loss'], label='Validation Loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend(loc='upper right')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Save the plot
    plot_path = os.path.join(PLOT_DIR, 'training_history.png')
    fig.savefig(plot_path)
    print(f"Training history plot saved to {plot_path}")


In [81]:
def evaluate_model(model, history, validation_generator):
    # Plot training history
    plot_training_history(history)
    
    # Evaluate model
    test_loss, test_accuracy = model.evaluate(validation_generator)
    print(f"\nTest accuracy: {test_accuracy:.4f}")
    print(f"Test loss: {test_loss:.4f}")
    
    # Generate predictions and confusion matrix
    predictions = model.predict(validation_generator)
    y_pred = (predictions > 0.5).astype(int)
    y_true = validation_generator.classes
    
    
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))


In [ ]:

try:
        # Set GPU memory growth
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        # Train model
    model, history, validation_generator = train_model()
        
    if history and history.history:
            # Evaluate results
        evaluate_model(model, history, validation_generator)
            
            # Save final model
        model.save('final_model.keras')
        print("\nTraining completed successfully!")
    else:
        print("Training did not complete successfully - no history object")
            
except Exception as e:
    print(f"\nAn error occurred in main execution: {str(e)}")